In [1]:
import mysql.connector

In [2]:
db_config = {
    "user": "tournament_user",
    "password": "tournament_pass",
    "host": "localhost",
    "database": "tournament"
}

In [3]:
select_first_names = "SELECT first_names.value FROM first_names WHERE first_names.language = 'Portuguese';"
select_last_names = "SELECT last_names.value FROM last_names WHERE last_names.language = 'Portuguese';"

In [4]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

In [5]:
cursor.execute(select_first_names)
br_first_names = cursor.fetchall()

In [6]:
cursor.execute(select_last_names)
br_last_names = cursor.fetchall()

In [7]:
cursor.execute(select_first_names.replace('Portuguese','Spanish'))
g_first_names = cursor.fetchall()

In [8]:
cursor.execute(select_last_names.replace('Portuguese', 'Spanish'))
g_last_names = cursor.fetchall()

In [9]:
select_clubs = "SELECT id, name FROM clubs;"

In [10]:
cursor.execute(select_clubs)
clubs = cursor.fetchall()

In [11]:
#create a list of players

from random import choice, randint, uniform

insert_contract = "INSERT INTO player_contracts (start, end, salary, club_id, player_id) VALUES(%s, %s, %s, %s, %s);"
insert_players = "INSERT INTO players VALUES(NULL, %s, %s, %s, %s, %s, %s, %s);"
select_last_players = "SELECT id FROM players ORDER BY(id) DESC LIMIT 30;"

for club in clubs:
    club_id = club[0]
    n_for_player = { 'GK': 3, 'DF': 9, 'MF': 11, 'AT': 7 }

    # for position, number_of_players_per_position
    for ps, n_ps in n_for_player.items():
        # create and insert players into database 
        gk = ['GK']
        df = ['LB', 'RB', 'CB']
        mf = ['DM', 'CM', 'RM', 'LM', 'AM' ]
        at = ['SS', 'WG', 'CF' ]

        for _ in range(n_ps):
            if ps == 'GK':
                position = 'GK'
                height = round(uniform(1.87, 1.99), 2)
            elif ps == 'DF':
                position = choice(df)
                height = round(uniform(1.80, 1.90), 2)
            elif ps == 'MF':
                position = choice(mf)
                height = round(uniform(1.60, 1.90), 2)
            elif ps == 'AT':
                position = choice(at)
                height = round(uniform(1.60, 1.95), 2)
            else:
                position = None

            weight = round(uniform(60.0, 90.9), 2)
            foot = choice(['R','L'])
            birth = str(randint(1984,2006))
    
            nationality = choice(['Brazil', 'Argentina', 'Colombia', 'Uruguay', 'Paraguay', 'Chile'])
            if nationality == 'Brazil':
                name = ' '.join([choice(br_first_names)[0], choice(br_last_names)[0]]) 
            else: 
                name = ' '.join([choice(g_first_names)[0], choice(g_last_names)[0]])
    
            cursor.execute(insert_players, [name, nationality, position, birth, height, weight, foot ])
                # end of player insertion 
        
        conn.commit()


    cursor.execute(select_last_players)
    last_players = cursor.fetchall()

    for player in last_players:
        cursor.execute(insert_contract, ['2022','2026', 100_000, club[0], player[0]])




In [12]:
conn.close()